# Covariance, Correlation, and Ordinary Least Squares Regression

For this lab, we will be using some randomly-generated data that I have made available on Github. 

In [ ]:
# load necessary packages using pacman

install.packages('pacman')

pacman::p_load(tidyverse)

In [ ]:
# ingest the data from github and store as "fake"
fake <- read.csv("https://raw.githubusercontent.com/bowendc/pol200_labs/refs/heads/main/fake.csv")

## Covariance and Correlation

Covariance and correlation are fairly easy to calculate in base R with built-in functions from the core **stats** package. But first, let's take a quick look at the relationship.

In [ ]:
# a really quick look at the relationship between x and y 
plot(fake$x, fake$y)

Now, let's calculate COV(XY).

In [ ]:
# the use = pairwise.complete.obs argument will keep any observation with
# non-missing data on both x and y. 
cov(fake$x, fake$y, use = "pairwise.complete.obs")

Again, covariance is hard to interpret. Instead, we typically standardize covariance by dividing by the standard deviation. When we do that, we get Pearson's *r* correlation coefficient. 

In [ ]:
cor(fake$x, fake$y, use = "pairwise.complete.obs")

We can save this or format by rounding:

In [ ]:
r <- cor(fake$x, fake$y, use = "pairwise.complete.obs")
r <- round(r, 3)
r

How about we check R's work by calculating *r* step by step ourselves? We can do so easily.

In [ ]:
# calculate the deviations of x and y from their means and divide by their standard deviations
fake$x_stand <- (fake$x - mean(fake$x, na.rm = TRUE)) / sd(fake$x, na.rm = TRUE)
fake$y_stand <- (fake$y - mean(fake$y, na.rm = TRUE)) / sd(fake$y, na.rm = TRUE)

# multiple standardized deviations together, observation by observation
fake$xy <- fake$x_stand * fake$y_stand

# check the data to see if it is working
head(fake)

# summarize to calculate final score
fake |> 
    filter(!is.na(xy)) |> 
    summarize(total = sum(xy),
              n = n()) |>
    mutate(r = total / (n - 1))


Hey look, this matches what we got from R's `cor()` function!

## Ordinary Least Squares

We will be doing a TON of OLS modeling in this class. For now, let's just go over the basics. We can conduct an OLS regression using `lm()` for "Linear Models." Here is the syntax:

In [ ]:
lm(y ~ x, data = fake)

We will typically want to store model output for working with later (predicting, calculating residuals, creating tables of regression results, etc).

In [ ]:
m1 <- lm(y ~ x, data = fake)

# summary() gives a quick print out of OLS
# results with some additional info, including the R2 score
summary(m1)

Since we've stored the model results, we can pull out those estimates when we need them. Below, let's store our coefficients as `a` and `b` for easy use.

In [ ]:
coefficients(m1)

# we need to use as.numeric() to get R to store the pulled
#   results as a number for calculating

a <- as.numeric(coefficients(m1)["(Intercept)"])
b <- as.numeric(coefficients(m1)["x"]) 
a
b

Now that we can access the model parameters easily, let's generate some predictions from our regression results! 

In [ ]:
# first, find min, max, mean, and sd of x:
xmin <- min(fake$x, na.rm = TRUE)
xmax <- max(fake$x, na.rm = TRUE)
xmean <- mean(fake$x, na.rm = TRUE)
xsd <- sd(fake$x, na.rm = TRUE)

# calculate the predicted values of y when x is at min, mean, and max values:
a + b*xmin
a + b*xmean
a + b*xmax

Remember, the coefficient for `x` tells us how `y` changes, on average, when `x` goes up 1 unit. But we can play around with other sizes of changes of `x`, simply by multiplying the coefficient by other relevant values. 

In [ ]:
# calculate what would happen to predicted value of y if x went up by 1 SD:
xsd * b

# calculate what would happen to predicted value of y if x moved from min to max values:
(xmax - xmin)*b